In [1]:
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
from pyscf.scf import cphf 
# ancora il modo migliore per mettere cariche frazionali sulle molecole, se vai a toccare mol.charges si incazza 
#  quando deve calcolare l' initial guess 
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords()/angstrom, deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [2]:
co=gto.Mole(atom="C 0 0 0 ;O 0 0 2.", basis= "sto-3g",units="Bohr",maxit=100)
hfco=scf.RHF(co)
hfco.scf()
dL=.001
hf1=fc(hfco,[dL,-dL])
hf1.scf()
hf2=fc(hfco,[-dL,dL])
hf2.scf()
dV=DeltaV(co,[dL,-dL])
h2=hfco.get_veff()
dh2=(hf1.get_veff()-hf2.get_veff())/2
J,K=hfco.get_jk()
g_ijkl=hfco.mol.intor('int2e_sph')
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2

C=hfco.mo_coeff
S=hfco.get_ovlp()
e=hfco.mo_energy
E=np.diag(e)
O=np.diag(hfco.mo_occ)
C1=hf1.mo_coeff
P=hfco.make_rdm1()
P1=hf1.make_rdm1()
P2=hf2.make_rdm1() 
dP=(P1-P2)/2
dP2=(P1-2*P+P2)/2/dL**2
dC=(abs(C1)*C/abs(C)-C)

Initialize <pyscf.gto.mole.Mole object at 0x7f81a3412d10> in <pyscf.scf.hf.RHF object at 0x7f81a39f8810>
<class 'pyscf.gto.mole.Mole'> does not have attributes  maxit units


converged SCF energy = -110.811052248023
converged SCF energy = -110.803479299976
converged SCF energy = -110.818627452765


In [3]:
G_cphf=g_ijkl.copy()
G_cphf=4*G_cphf -G_cphf.swapaxes(3,1)-G_cphf.swapaxes(1,2)
G_cphf=G_cphf@C
G_cphf=(G_cphf.swapaxes(2,3)@C).swapaxes(3,2)
G_cphf=(G_cphf.swapaxes(1,3)@C).swapaxes(3,1)
G_cphf=(G_cphf.swapaxes(0,3)@C).swapaxes(3,0)
A=G_cphf.copy()
for i in range(co.nao):
    for a in range(co.nao):
        if e[i] !=e[a] :
            A[i,a,:,:]/=(e[a]-e[i])

In [4]:
dV_mo=C.T@dV@C

U_0= dV_mo.copy()
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            U_0[i][j]/=(e[j]-e[i])
Us=[]
Us.append(U_0)
for k in range(10):
    U=np.sum(u for u in Us)
    dP_app=C@(U@O-O@U)@C.T
    print(np.linalg.norm(dP_app-dP))
    Us.append(np.einsum('ijkl,kl->ij',A[:,:,co.nelec[0]:,:co.nelec[0]],Us[k][co.nelec[0]:,:co.nelec[0]])) # approximate value of U
    U=np.sum(u for u in Us)
    dP_app=C@(U@O-O@U)@C.T
    
print(np.linalg.norm(dP_app-dP))

0.00039868625143894804
0.00013173347206543834
7.681573703356613e-05
4.475225066090898e-05
2.6357152978804786e-05
1.549265259427724e-05
9.11425504589184e-06
5.366134288911784e-06
3.1645709419431844e-06
1.8715606515800586e-06
1.1124340604063202e-06


/home/giorgiod/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  # This is added back by InteractiveShellApp.init_path()
/home/giorgiod/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  from ipykernel import kernelapp as app


In [5]:
dEnn=-1/2.
print("true target energy: ",fc(hfco,[1,-1]).scf())
print("CO energy = ",hfco.e_tot)
print("APDFT1 energy =", hfco.e_tot+dEnn+np.trace(P.dot(dV/dL)))
print("APDFT2 energy =", hfco.e_tot+dEnn+np.trace((P+0.5*dP/dL).dot(dV/dL)))
print("APDFT2 energy approx.=", hfco.e_tot+dEnn+np.trace((P+ 0.5*dP_app/dL).dot(dV/dL)))
print("APDFT3 energy =", hfco.e_tot+dEnn+np.trace((P+0.5*dP/dL+(1/6)*dP2).dot(dV/dL)))


converged SCF energy = -104.823560975956
true target energy:  -104.8235609759558
CO energy =  -110.81105224802306
APDFT1 energy = -104.26615335785836
APDFT2 energy = -105.12993171823632
APDFT2 energy approx.= -105.12988869744264
APDFT3 energy = -104.9945636667493


In [6]:
np.trace( dP_app/dL@dV/dL)

-1.7274706791685661

In [7]:
4*np.trace(U[:co.nelec[0],co.nelec[0]:]@dV_mo[co.nelec[0]:,:co.nelec[0]])/dL**2

1.7274706791685661

$$ \frac {\partial^2 E}{\partial Z^2}= 4 \sum_a \sum_i U_{ia} \Delta V^{mo}_{ai}
$$